## Import Dependencies

In [1]:
import sys
import math
import numpy as np
import trimesh
from importlib import reload

from machina import HexMachina
from optimization import *
from singularity import *
from utils import *

import parametrization
import extraction

ImportError: /home/dnkrtz/anaconda3/lib/python3.5/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.21' not found (required by /home/dnkrtz/anaconda3/lib/python3.5/site-packages/MeshPy-2016.1-py3.5-linux-x86_64.egg/meshpy/_tetgen.cpython-35m-x86_64-linux-gnu.so)

## Generate Tetrahedral Mesh
First, the triangle mesh representing the geometry is imported. Based on this input mesh, a volumetric tetrahedral mesh is generated and its boundary surface is extracted. Next, the curvatures and normals of this surface are computed, and some additional topological information is computed (i.e. dual).

In [4]:
print('Reading triangle mesh...', end=" ")
sys.stdout.flush()
tri_mesh = trimesh.load_mesh('../io/cylinder.stl')
say_ok()

# Instantiate tetrahedral mesh
print('Generating tetrahedral mesh...', end=" ")
sys.stdout.flush()
machina = HexMachina(tri_mesh, max_vol = 1)
# Output tetrahedral mesh
vtk_tetmesh(machina.tet_mesh, 'tet_mesh')
say_ok()

# Construct boundary surface of tetrahedral mesh.
print('Computing surface curvatures/normals...', end=" ")
sys.stdout.flush()
# Compute face and vertex normals.
machina.surf_mesh.compute_normals()
# Compute principal curvatures and directions.
machina.surf_mesh.compute_curvatures()
# Output curvature crossfield to .vtk file.
vtk_curvature(machina.surf_mesh, 'curvature')
say_ok()

# Compute the tetrahedral one-rings of the mesh.
print('Extracting dual voronoi...', end=" ")
sys.stdout.flush()
machina.compute_dual()
say_ok()

Reading triangle mesh...  OK. 
Generating tetrahedral mesh...  OK. 
Computing surface curvatures/normals...  OK. 
Extracting dual voronoi...  OK. 


## Construct Framefield

In [5]:
# Construct 3D frame field as an array of (U, V, W) frames.
# This field is parallel to the tet list (i.e. each tet has a frame).
print('Initializing framefield...', end=" ")
sys.stdout.flush()
# Construct frame field.
machina.init_framefield()
say_ok()

# Optimize 3D frame field by L-BFGS minimization.
print('Optimizing framefield...')
machina.optimize_framefield()
say_ok()

# Output frame field to .vtk file.
vtk_framefield(machina.frames, 'field')

Initializing framefield...  OK. 
Optimizing framefield...
 OK. 


## Pre-process

In [6]:
# Determine the singular edges of the framefield.      
print("Computing singular graph...", end=" ")
sys.stdout.flush()
singular_vertices = singular_graph(machina)[2]
say_ok()

Computing singular graph...  OK. 


## Parametrize

In [ ]:
reload(parametrization)

print("Parametrizing volume...")
uvw_map = parametrization.parametrize_volume(machina, singular_vertices, 3.0)
say_ok()

Parametrizing volume...
Computing laplacian and constraints...
Conjugate Gradient... (Round 1) 

## Extract hex

In [22]:
reload(extraction)

print("Extracting hexahedrons...", end=" ")
iso_pts = extraction.extract_isolines(machina, uvw_map)
say_ok()

Extracting hexahedrons... 

/home/dnkrtz/Projects/HexMachina/hexmachina/extraction.py:95: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if iso_pt != None:


 OK. 
